# Sample 02: OpenAI SDK with Python (Jupyter Notebook)

This notebook demonstrates how to use the OpenAI Python SDK to connect to AI Foundry models. We'll explore basic chat completions, streaming responses, and different configuration options.

## Prerequisites
- Python environment with required packages installed
- `.env` file configured with AI Foundry credentials
- OpenAI Python SDK installed

Let's get started!

## 1. Environment Setup and Configuration

First, let's verify our environment and install any missing packages. The required packages should already be installed if you followed the main README setup instructions.

In [ ]:
# Verify required packages are installed
import sys
import subprocess

def check_package(package_name):
    try:
        __import__(package_name)
        print(f"✅ {package_name} is installed")
        return True
    except ImportError:
        print(f"❌ {package_name} is not installed")
        return False

# Check required packages
packages = ['openai', 'dotenv']
all_installed = all(check_package(pkg) for pkg in packages)

if all_installed:
    print("\n🎉 All required packages are installed!")
else:
    print("\n⚠️  Some packages are missing. Please run: pip install -r requirements.txt")

## 2. Import Required Libraries

Now let's import all the libraries we'll need for this notebook.

In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import time

print("📦 Libraries imported successfully!")

## 3. Load Environment Variables

Load our AI Foundry configuration from the `.env` file. Make sure you've copied `.env.example` to `.env` and filled in your actual credentials.

In [ ]:
# Load environment variables from .env file
# Note: The .env file should be in the root directory (../../.env from this notebook)
load_dotenv('../../.env')

# Get configuration from environment variables
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
api_version = os.getenv('AZURE_OPENAI_API_VERSION')

# Verify configuration is loaded
print("🔧 Configuration Status:")
print(f"Endpoint: {'✅ Loaded' if azure_endpoint else '❌ Missing'}")
print(f"Deployment: {'✅ Loaded' if deployment_name else '❌ Missing'}")
print(f"API Version: {'✅ Loaded' if api_version else '❌ Missing'}")

if all([azure_endpoint, deployment_name, api_version]):
    print("\n🎉 All configuration loaded successfully!")
else:
    print("\n⚠️  Some configuration is missing. Please check your .env file.")

## 4. Initialize OpenAI Client

Create the Azure OpenAI client using our configuration.

In [ ]:
# Initialize the Azure OpenAI client with DefaultAzureCredential
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)
client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    azure_ad_token_provider=token_provider,
    api_version=api_version
)

print("🚀 Azure OpenAI client initialized successfully with DefaultAzureCredential!")
print(f"Connected to: {azure_endpoint}")
print(f"Using deployment: {deployment_name}")

## 5. Basic Chat Completion

Let's start with a simple chat completion request. This is the most basic way to interact with the AI model.

In [ ]:
# Simple chat completion example

def basic_chat_completion(user_message):
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": user_message}
            ],
            max_tokens=150,
            temperature=0.7
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        return f"Error: {str(e)}"

# Test the function
user_input = "Hello! Can you tell me a short joke?"
print("👤 User:", user_input)
print("🤖 AI:", basic_chat_completion(user_input))

## 6. Conversation with History

Now let's demonstrate how to maintain conversation context by including message history.

In [ ]:
# Conversation with history example
def chat_with_history(messages):
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=150,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Create a conversation with history
conversation = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Paris."},
    {"role": "user", "content": "What is the population of that city?"}
]

print("📚 Conversation History:")
for i, message in enumerate(conversation[1:], 1):  # Skip system message for display
    role_emoji = "👤" if message["role"] == "user" else "🤖"
    print(f"{role_emoji} {message['role'].title()}: {message['content']}")

# Get response with context
print("\n🤖 AI:", chat_with_history(conversation))

## 7. Temperature Variations

Temperature controls the randomness of the AI's responses. Let's compare different temperature settings with the same prompt.

In [ ]:
# Temperature comparison
def chat_with_temperature(user_message, temperature):
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": "You are a helpful AI assistant. Keep your response to one sentence."},
                {"role": "user", "content": user_message}
            ],
            max_tokens=100,
            temperature=temperature
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

# Test the same prompt with different temperatures, running each twice
# The prompt is open enough for variety at high temp, but focused enough for consistency at low temp
prompt = "Suggest a creative name for a coffee shop."
temperatures = [
    (0.1, "Low temperature (more consistent)"),
    (1.0, "High temperature (more varied)")
]

print("👤 User:", prompt)
print("\n" + "="*50)

for temp, description in temperatures:
    print(f"\n🌡️ Temperature {temp} - {description}")
    for run in range(1, 3):
        print(f"\n  Run {run}:")
        print(f"  🤖 AI:", chat_with_temperature(prompt, temp))

## 8. Interactive Playground

Use this cell to experiment with your own prompts and settings. Modify the variables below and run the cell to test different configurations.

In [ ]:
# Interactive playground - modify these variables and run the cell
YOUR_SYSTEM_MESSAGE = "You are a helpful AI assistant."
YOUR_USER_MESSAGE = "Tell me an interesting fact about space."
YOUR_TEMPERATURE = 0.7
YOUR_MAX_TOKENS = 150

# Run your custom chat completion
try:
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": YOUR_SYSTEM_MESSAGE},
            {"role": "user", "content": YOUR_USER_MESSAGE}
        ],
        max_tokens=YOUR_MAX_TOKENS,
        temperature=YOUR_TEMPERATURE
    )
    
    print("🎮 Your Custom Chat:")
    print(f"👤 User: {YOUR_USER_MESSAGE}")
    print(f"🤖 AI: {response.choices[0].message.content}")
    print(f"\n📊 Settings: Temperature={YOUR_TEMPERATURE}, Max Tokens={YOUR_MAX_TOKENS}")
    print(f"📈 Usage: {response.usage.total_tokens} tokens used")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")

## 9. Summary and Next Steps

🎉 **Congratulations!** You've successfully learned how to:

1. ✅ Set up the OpenAI Python SDK with Azure OpenAI
2. ✅ Load configuration from environment variables
3. ✅ Make basic chat completion requests
4. ✅ Handle conversation history and context
5. ✅ Experiment with temperature settings
6. ✅ Create your own interactive playground

### Key Takeaways:
- **Environment Variables**: Keep your credentials secure by using `.env` files
- **Temperature**: Low (0.1) for factual, high (0.9) for creative responses
- **Context**: Include message history to maintain conversation flow
- **Error Handling**: Always wrap API calls in try-catch blocks

### Next Steps:
- Explore the other samples in this repository
- Try different system messages and prompts
- Experiment with other OpenAI parameters like `top_p`, `frequency_penalty`, etc.
- Build your own applications using these patterns

Happy coding! 🚀